In [13]:
import pandas as pd

In [14]:
def count_hallucinations_extract_generated_results(df,generated_text_col,instruction_lang):
    hallucination_flag = []
    actual_label = []
    
    for index, row in df.iterrows(): 
        i = row['Sentence']
        if instruction_lang == "English":
            instruction = "The following statement is in English language: " + i + ". Is the given statement hateful? Answer with a Yes or No."
        elif instruction_lang == "Arabic":
            instruction = "العبارة التالية مكتوبه باللغة العربية: " + i + ". هل العبارة السابقة تحتوي علي خطاب كراهية؟ اجب فقط بنعم او لا. "
        elif instruction_lang == "German":
            instruction = "Die folgende Erklärung ist in deutscher Sprache:" + i + "Ist die gegebene Aussage hasserfüllt? Antworten Sie mit Ja oder Nein."
        elif instruction_lang == "English-Arabic":
            instruction = "The following statement is in Arabic language: " + i + ". Is the given statement hateful? Answer with a Yes or No."
        else:
            instruction = "The following statement is in German language: " + i + ". Is the given statement hateful? Answer with a Yes or No."
            
        generated_text = row[generated_text_col]
        
        #print(instruction)
        #print(generated_text)
        if instruction in generated_text:
            new_outcome = generated_text.replace(instruction,'')

            if "Yes" in new_outcome or "No" in new_outcome or "لا" in new_outcome or "نعم" in new_outcome or "Nein" in new_outcome or "Ja" in new_outcome or "hateful" in new_outcome or "كراهية" in new_outcome or "hasserfüllt" in new_outcome:
                hallucination_flag.append("No-Hallucination")
                if "Yes" in new_outcome or "نعم" in new_outcome or "Ja" in new_outcome or "hateful" in new_outcome or "كراهية" in new_outcome or "hasserfüllt" in new_outcome:
                    actual_label.append(1)
                else:
                    actual_label.append(0)
            else:
                hallucination_flag.append("Hallucination")
                actual_label.append(-1)
        else:
            print("instruction is not in the generted text") 
            hallucination_flag.append("Hallucination")
            actual_label.append(-1)
    df ["Hallucination"] = hallucination_flag
    df ["actual_model_prediction"] = actual_label
    return df

In [15]:
Model_name = "Bloomz"
prediction_col_name = "bloomz-7b1_HGL"

In [16]:
female_arabic_results = pd.read_csv("../../../Arabic_LMs/MSA/Decoder_only/"+Model_name+"/Female_IFM_HSD_"+Model_name+"_arabic_instructions.csv")
female_englsih_results = pd.read_csv("../../../Arabic_LMs/MSA/Decoder_only/"+Model_name+"/Female_IFM_HSD_"+Model_name+"_English_instructions.csv")

male_arabic_results = pd.read_csv("../../../Arabic_LMs/MSA/Decoder_only/"+Model_name+"/Male_IFM_HSD_"+Model_name+"_arabic_instructions.csv")
male_englsih_results = pd.read_csv("../../../Arabic_LMs/MSA/Decoder_only/"+Model_name+"/Male_IFM_HSD_"+Model_name+"_English_instructions.csv")

NB_arabic_results = pd.read_csv("../../../Arabic_LMs/MSA/Decoder_only/"+Model_name+"/Non_Binary_IFM_HSD_"+Model_name+"_arabic_instructions.csv")
NB_englsih_results = pd.read_csv("../../../Arabic_LMs/MSA/Decoder_only/"+Model_name+"/Non_Binary_IFM_HSD_"+Model_name+"_English_instructions.csv")

In [17]:
female_arabic_results = count_hallucinations_extract_generated_results(female_arabic_results, prediction_col_name, "Arabic")
female_englsih_results = count_hallucinations_extract_generated_results(female_englsih_results, prediction_col_name, "English-Arabic")


male_arabic_results = count_hallucinations_extract_generated_results(male_arabic_results, prediction_col_name, "Arabic")
male_englsih_results = count_hallucinations_extract_generated_results(male_englsih_results, prediction_col_name, "English-Arabic")


NB_arabic_results = count_hallucinations_extract_generated_results(NB_arabic_results, prediction_col_name, "Arabic")
NB_englsih_results = count_hallucinations_extract_generated_results(NB_englsih_results, prediction_col_name, "English-Arabic")

In [18]:
female_arabic_results.to_csv("./Clean_results/"+Model_name+"/Female_IFM_HSD_"+Model_name+"_arabic_instructions.csv", index=False)
female_englsih_results.to_csv("./Clean_results/"+Model_name+"/Female_IFM_HSD_"+Model_name+"_english_instructions.csv", index=False)

male_arabic_results.to_csv("./Clean_results/"+Model_name+"/Male_IFM_HSD_"+Model_name+"_arabic_instructions.csv", index=False)
male_englsih_results.to_csv("./Clean_results/"+Model_name+"/Male_IFM_HSD_"+Model_name+"_english_instructions.csv", index=False)

NB_arabic_results.to_csv("./Clean_results/"+Model_name+"/NB_IFM_HSD_"+Model_name+"_arabic_instructions.csv", index=False)
NB_englsih_results.to_csv("./Clean_results/"+Model_name+"/NB_IFM_HSD_"+Model_name+"_english_instructions.csv", index=False)